In [1]:
from sqlalchemy import extract, desc
import datetime
import models as db
import time
import MetaTrader5 as mt5
import pandas as pd

In [2]:
if not mt5.initialize():
    print("initialize() failed")
    mt5.shutdown()


In [51]:
def get_book_and_return_first_line(symb):
    #get ASK and BID price and volume
    #returning BookInfo
    mt5.market_book_add(symb)
    time.sleep(0.1)
    items = mt5.market_book_get(symb)
    mt5.market_book_release(symb)  
    if len(items) > 0:
        for l in range(0,len(items)):
            if l > 0:
                if items[l].type != items[l-1].type:
                    return {'bid': items[l], 'ask': items[l-1]}

# bid - [BookInfo(type=2, price=1.02, volume=41100, volume_dbl=41100.0),
# ask - [BookInfo(type=1, price=1.07, volume=41100, volume_dbl=41100.0)]

def update_quotes(stock_class=db.PETR4, stock_class_OPTIONS=db.PETR4_OPTIONS, just_last_update = True):
    due_dates = db.session.query(stock_class.due_date).distinct().filter(stock_class.due_date > datetime.datetime.today()).limit(5)
    due_dates = list(due_dates)

    month_deadline = due_dates[0].due_date
    next_month_dealine = due_dates[1].due_date
    after_next_month_dealine = due_dates[2].due_date
    long_due = due_dates[3].due_date

    this_month = datetime.datetime.today().date()
    diff_betwen_dates = month_deadline - this_month

    if diff_betwen_dates.days < 5:
        month_deadline = next_month_dealine
        next_month_dealine = after_next_month_dealine
        after_next_month_dealine = long_due

    last_date = db.session.query(stock_class_OPTIONS).filter(stock_class_OPTIONS.due_date == month_deadline).order_by(desc(stock_class_OPTIONS.updated_at)).first()
    
    if just_last_update == True:
        month_last_ticks = db.session.query(stock_class_OPTIONS).filter(stock_class_OPTIONS.updated_at == last_date.updated_at, stock_class_OPTIONS.due_date == month_deadline).order_by(stock_class_OPTIONS.strike).all()
        next_month_last_ticks = db.session.query(stock_class_OPTIONS).filter(stock_class_OPTIONS.updated_at == last_date.updated_at, stock_class_OPTIONS.due_date == next_month_dealine).order_by(stock_class_OPTIONS.strike).all()
        after_next_month_last_ticks = db.session.query(stock_class_OPTIONS).filter(stock_class_OPTIONS.updated_at == last_date.updated_at, stock_class_OPTIONS.due_date == after_next_month_dealine).order_by(stock_class_OPTIONS.strike).all()
    else:
        month_last_ticks = db.session.query(stock_class_OPTIONS).filter(stock_class_OPTIONS.due_date == month_deadline).order_by(stock_class_OPTIONS.updated_at,stock_class_OPTIONS.strike).all()
        next_month_last_ticks = db.session.query(stock_class_OPTIONS).filter(stock_class_OPTIONS.due_date == next_month_dealine).order_by(stock_class_OPTIONS.updated_at,stock_class_OPTIONS.strike).all()
        after_next_month_last_ticks = db.session.query(stock_class_OPTIONS).filter(stock_class_OPTIONS.due_date == after_next_month_dealine).order_by(stock_class_OPTIONS.updated_at,stock_class_OPTIONS.strike).all()
        
        #for tick in month_last_ticks:
         #   print(tick.updated_at, tick.timestamp_option, tick.option_name, tick.strike, tick.deal_type_zone, tick.stock_price, tick.bid, tick.ask, tick.last_tick)        
    
    return [month_last_ticks,next_month_last_ticks,after_next_month_last_ticks]
       

In [52]:
def reorder_columns(columns, first_cols=[], last_cols=[], drop_cols=[]):
    columns = list(set(columns) - set(first_cols))
    columns = list(set(columns) - set(drop_cols))
    columns = list(set(columns) - set(last_cols))
    new_order = first_cols + columns + last_cols
    return new_order

In [53]:
updated_ticks = update_quotes(stock_class=db.PETR4,stock_class_OPTIONS=db.PETR4_OPTIONS,just_last_update=False)
this_due = updated_ticks[0]
next_due = updated_ticks[1]
after_next_due = updated_ticks[1]

In [54]:
options = []
for i in this_due:
    options.append(i.__dict__)

df_options = pd.DataFrame.from_dict(options)
df_options = df_options.drop(columns=['_sa_instance_state','timestamp_option','days_to_due_date'])
df_options['ratio'] = df_options['last_tick'] - df_options['last_tick'].shift(1)
df_options['ratio_2'] = df_options['ratio'] - df_options['ratio'].shift(1)
df_options = df_options[['option_name', 'strike', 'bid', 'ask', 'last_tick','ratio','ratio_2','stock_price','deal_type_zone','updated_at']]

In [55]:

df_options

,option_name,strike,bid,ask,last_tick,ratio,ratio_2,stock_price,deal_type_zone,updated_at
0,PETRH269,26.21,2.46,2.53,2.38,NaN,NaN,28.09,ITM,2021-07-14 10:09:44.647
1,PETRH254,26.46,2.28,2.34,2.19,-0.19,NaN,28.09,ITM,2021-07-14 10:09:44.647
2,PETRH274,26.71,2.11,2.16,2.02,-0.17,0.02,28.09,ITM,2021-07-14 10:09:44.647
3,PETRH276,26.96,1.92,1.98,1.83,-0.19,-0.02,28.09,ITM,2021-07-14 10:09:44.647
4,PETRH261,27.21,1.76,1.82,1.81,-0.02,0.17,28.09,ITM,2021-07-14 10:09:44.647
...,...,...,...,...,...,...,...,...,...,...
29661,PETRH243,28.96,0.69,0.71,0.71,-0.05,0.06,27.67,OTM,2021-07-14 17:25:23.300
29662,PETRH299,29.21,0.60,0.61,0.61,-0.10,-0.05,27.67,OTM,2021-07-14 17:25:23.300
29663,PETRH251,29.46,0.46,0.50,0.50,-0.11,-0.01,27.67,OTM,2021-07-14 17:25:23.300
29664,PETRH304,29.71,0.49,0.50,0.49,-0.01,0.10,27.67,OTM,2021-07-14 17:25:23.300


In [47]:
df_options2 = df_options[['option_name', 'strike', 'bid', 'ask', 'last_tick','ratio','stock_price','deal_type_zone','updated_at']]
df_options2

,option_name,strike,bid,ask,last_tick,ratio,stock_price,deal_type_zone,updated_at
0,PETRH269,26.21,2.46,2.53,2.38,NaN,28.09,ITM,2021-07-14 10:09:44.647
1,PETRH254,26.46,2.28,2.34,2.19,-0.19,28.09,ITM,2021-07-14 10:09:44.647
2,PETRH274,26.71,2.11,2.16,2.02,-0.17,28.09,ITM,2021-07-14 10:09:44.647
3,PETRH276,26.96,1.92,1.98,1.83,-0.19,28.09,ITM,2021-07-14 10:09:44.647
4,PETRH261,27.21,1.76,1.82,1.81,-0.02,28.09,ITM,2021-07-14 10:09:44.647
...,...,...,...,...,...,...,...,...,...
29661,PETRH243,28.96,0.69,0.71,0.71,-0.05,27.67,OTM,2021-07-14 17:25:23.300
29662,PETRH299,29.21,0.60,0.61,0.61,-0.10,27.67,OTM,2021-07-14 17:25:23.300
29663,PETRH251,29.46,0.46,0.50,0.50,-0.11,27.67,OTM,2021-07-14 17:25:23.300
29664,PETRH304,29.71,0.49,0.50,0.49,-0.01,27.67,OTM,2021-07-14 17:25:23.300
